In [275]:
'''
使用VS Code+jupyter notebook编译
使用Nvidia RTX3060 GPU进行训练
作者:张伟业
'''
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(torch.cuda.is_available())
print(torch.__version__)
device

True
2.1.1+cu121


device(type='cuda', index=0)

导入所需要的包

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np
import urllib.request
import tarfile
import os
import os.path as osp
import matplotlib.pyplot as plt
from PIL import Image
import random
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau


随机种子设定

In [277]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

定义 CNN 网络结构

In [278]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 4, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


定义深度卷积网络结构

In [279]:
class DeepConvNet(nn.Module):
    def __init__(self):
        super(DeepConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU(inplace=True)

        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU(inplace=True)

        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.relu6 = nn.ReLU(inplace=True)

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))
        x = self.relu4(self.bn4(self.conv4(x)))
        x = self.relu5(self.bn5(self.conv5(x)))
        x = self.relu6(self.bn6(self.conv6(x)))

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

定义 CNN 加 Rsidual Block 的新网络结构

In [280]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.downsample = None

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResidualNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResidualNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.residual_block1 = ResidualBlock(32, 32)
        self.residual_block2 = ResidualBlock(32, 64, stride=2)
        self.residual_block3 = ResidualBlock(64, 128, stride=2)
        self.residual_block4 = ResidualBlock(128, 256, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))

        x = self.residual_block1(x)
        x = self.residual_block2(x)
        x = self.residual_block3(x)
        x = self.residual_block4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

超参数设定

In [281]:
# Training
BATCH_SIZE = 128  # 每个训练批次中包含的样本数量
NUM_EPOCHS = 10  # 训练迭代的总轮数
EVAL_INTERVAL = 2  # 用于设定多少个 epoch 后进行模型性能评估
SAVE_DIR = './log'  # 保存训练日志和模型检查点的目录

# Optimizer
LEARNING_RATE = 1e-1  # 学习率，用于控制权重更新的步长
MOMENTUM = 0.9  # 动量参数，用于加速权重更新
STEP = 10  # 学习率调度的步数
GAMMA = 0.1  # 学习率调度的衰减率 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

数据集加载与处理

In [282]:
# 定义数据集的URL和存储路径
url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
data_dir = './dataset-cifar10'

# 创建存储路径（如果不存在）
os.makedirs(data_dir, exist_ok=True)

# 下载并解压缩数据集
filename = url.split('/')[-1]
filepath = os.path.join(data_dir, filename)

if not os.path.exists(filepath):
    urllib.request.urlretrieve(url, filepath)
    tar = tarfile.open(filepath, 'r:gz')
    tar.extractall(data_dir)
    tar.close()


In [283]:
# 定义训练数据的图像预处理操作：
transform_cifar10_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 定义测试数据的图像预处理操作：
transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#定义训练和测试的数据集以及数据加载器
train_set = datasets.CIFAR10(root=data_dir, train=True,
                                        download=False, transform=transform_cifar10_train)

train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE,
                                            shuffle=True, num_workers=2)

test_set = datasets.CIFAR10(root=data_dir, train=False,
                                        download=False, transform=transform_cifar10_test)

test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE,
                                            shuffle=False, num_workers=2)


In [284]:
def train_batch(model, image, target):
    output = model(image)
    loss_fn =nn.CrossEntropyLoss()
    loss = loss_fn(output , target)
    return output, loss

def test_batch(model, image, target):
    output = model(image)
    loss_fn = nn.CrossEntropyLoss()
    loss = loss_fn(output, target)
    return output, loss

模型选择与训练-CNN

In [285]:
torch.backends.cudnn.deterministic = True
model = ConvNet()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [286]:
for epoch in range(NUM_EPOCHS):
    model.train()  # 设置模型为训练模式
    torch.cuda.empty_cache()  # 清空GPU缓存

    # 训练阶段
    running_cls_loss = 0.0  # 用于存储每个batch的训练损失
    running_cls_corrects = 0  # 用于存储每个batch的训练正确预测的样本数

    # 遍历训练数据集的每个batch
    for batch_idx, (image, target) in enumerate(train_dataloader):

        image = image.to(device)  # 将图像数据移动到GPU（如果可用）
        target = target.to(device)  # 将标签数据移动到GPU（如果可用）

        # 训练模型并获取输出和损失
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)  # 获取预测的类别

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')  # 检查损失是否为NaN
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 使用优化器更新模型参数
        optimizer.zero_grad()  # 清空梯度

    # 计算当前epoch的平均损失和准确度
    epoch_loss = running_cls_loss / len(train_set)
    epoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch + 1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


    # 调整学习率
    scheduler.step()  # 调整学习率策略

    # 测试阶段
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()  # 设置模型为评估模式

        val_loss = 0.0  # 用于存储测试损失
        val_corrects = 0  # 用于存储测试正确预测的样本数

        # 遍历测试数据集的每个batch
        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)  # 将图像数据移动到GPU（如果可用）
            target = target.to(device)  # 将标签数据移动到GPU（如果可用）

            # 测试模型并获取输出和损失
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)  # 获取预测的类别

            val_loss += loss.item()
            val_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        val_loss = val_loss / len(test_set)  # 计算平均测试损失
        val_acc = val_corrects.double() / len(test_set)  # 计算测试准确度
        print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # 在最后一个epoch保存模型
        if (epoch + 1) == NUM_EPOCHS:

            state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch + 1),
            }

            # 检查目录是否存在，如果不存在则创建
            if not os.path.exists(SAVE_DIR):
                os.makedirs(SAVE_DIR)

            # 保存模型状态
            torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch + 1))))

Epoch: 1/10 Train Loss: 0.0138 Acc: 0.3536
Epoch: 2/10 Train Loss: 0.0128 Acc: 0.4115
Begin test......
Test Loss: 0.0123 Acc: 0.4498
Epoch: 3/10 Train Loss: 0.0125 Acc: 0.4322
Epoch: 4/10 Train Loss: 0.0124 Acc: 0.4368
Begin test......
Test Loss: 0.0130 Acc: 0.4108
Epoch: 5/10 Train Loss: 0.0122 Acc: 0.4465
Epoch: 6/10 Train Loss: 0.0119 Acc: 0.4626
Begin test......
Test Loss: 0.0123 Acc: 0.4632
Epoch: 7/10 Train Loss: 0.0120 Acc: 0.4561
Epoch: 8/10 Train Loss: 0.0121 Acc: 0.4559
Begin test......
Test Loss: 0.0128 Acc: 0.4437
Epoch: 9/10 Train Loss: 0.0120 Acc: 0.4646
Epoch: 10/10 Train Loss: 0.0120 Acc: 0.4617
Begin test......
Test Loss: 0.0129 Acc: 0.4356


模型选择与训练-DeepCNN

In [287]:
torch.backends.cudnn.deterministic = True
model = DeepConvNet()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [288]:
for epoch in range(NUM_EPOCHS):
    model.train()  # 设置模型为训练模式
    torch.cuda.empty_cache()  # 清空GPU缓存

    # 训练阶段
    running_cls_loss = 0.0  # 用于存储每个batch的训练损失
    running_cls_corrects = 0  # 用于存储每个batch的训练正确预测的样本数

    # 遍历训练数据集的每个batch
    for batch_idx, (image, target) in enumerate(train_dataloader):

        image = image.to(device)  # 将图像数据移动到GPU（如果可用）
        target = target.to(device)  # 将标签数据移动到GPU（如果可用）

        # 训练模型并获取输出和损失
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)  # 获取预测的类别

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')  # 检查损失是否为NaN
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 使用优化器更新模型参数
        optimizer.zero_grad()  # 清空梯度

    # 计算当前epoch的平均损失和准确度
    epoch_loss = running_cls_loss / len(train_set)
    epoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch + 1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


    # 调整学习率
    scheduler.step()  # 调整学习率策略

    # 测试阶段
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()  # 设置模型为评估模式

        val_loss = 0.0  # 用于存储测试损失
        val_corrects = 0  # 用于存储测试正确预测的样本数

        # 遍历测试数据集的每个batch
        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)  # 将图像数据移动到GPU（如果可用）
            target = target.to(device)  # 将标签数据移动到GPU（如果可用）

            # 测试模型并获取输出和损失
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)  # 获取预测的类别

            val_loss += loss.item()
            val_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        val_loss = val_loss / len(test_set)  # 计算平均测试损失
        val_acc = val_corrects.double() / len(test_set)  # 计算测试准确度
        print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # 在最后一个epoch保存模型
        if (epoch + 1) == NUM_EPOCHS:

            state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch + 1),
            }

            # 检查目录是否存在，如果不存在则创建
            if not os.path.exists(SAVE_DIR):
                os.makedirs(SAVE_DIR)

            # 保存模型状态
            torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch + 1))))

Epoch: 1/10 Train Loss: 0.0122 Acc: 0.4224
Epoch: 2/10 Train Loss: 0.0090 Acc: 0.5840
Begin test......
Test Loss: 0.0114 Acc: 0.5184
Epoch: 3/10 Train Loss: 0.0075 Acc: 0.6586
Epoch: 4/10 Train Loss: 0.0065 Acc: 0.7071
Begin test......
Test Loss: 0.0109 Acc: 0.5761
Epoch: 5/10 Train Loss: 0.0058 Acc: 0.7404
Epoch: 6/10 Train Loss: 0.0051 Acc: 0.7720
Begin test......
Test Loss: 0.0086 Acc: 0.6578
Epoch: 7/10 Train Loss: 0.0047 Acc: 0.7912
Epoch: 8/10 Train Loss: 0.0042 Acc: 0.8130
Begin test......
Test Loss: 0.0054 Acc: 0.7632
Epoch: 9/10 Train Loss: 0.0038 Acc: 0.8304
Epoch: 10/10 Train Loss: 0.0034 Acc: 0.8479
Begin test......
Test Loss: 0.0060 Acc: 0.7584


模型选择与训练-ResNet

In [289]:
torch.backends.cudnn.deterministic = True
model = ResidualNet()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [290]:
for epoch in range(NUM_EPOCHS):
    model.train()  # 设置模型为训练模式
    torch.cuda.empty_cache()  # 清空GPU缓存

    # 训练阶段
    running_cls_loss = 0.0  # 用于存储每个batch的训练损失
    running_cls_corrects = 0  # 用于存储每个batch的训练正确预测的样本数

    # 遍历训练数据集的每个batch
    for batch_idx, (image, target) in enumerate(train_dataloader):

        image = image.to(device)  # 将图像数据移动到GPU（如果可用）
        target = target.to(device)  # 将标签数据移动到GPU（如果可用）

        # 训练模型并获取输出和损失
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)  # 获取预测的类别

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')  # 检查损失是否为NaN
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 使用优化器更新模型参数
        optimizer.zero_grad()  # 清空梯度

    # 计算当前epoch的平均损失和准确度
    epoch_loss = running_cls_loss / len(train_set)
    epoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch + 1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


    # 调整学习率
    scheduler.step()  # 调整学习率策略

    # 测试阶段
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()  # 设置模型为评估模式

        val_loss = 0.0  # 用于存储测试损失
        val_corrects = 0  # 用于存储测试正确预测的样本数

        # 遍历测试数据集的每个batch
        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)  # 将图像数据移动到GPU（如果可用）
            target = target.to(device)  # 将标签数据移动到GPU（如果可用）

            # 测试模型并获取输出和损失
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)  # 获取预测的类别

            val_loss += loss.item()
            val_corrects += torch.sum(preds == target.data)  # 统计正确预测的样本数

        val_loss = val_loss / len(test_set)  # 计算平均测试损失
        val_acc = val_corrects.double() / len(test_set)  # 计算测试准确度
        print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # 在最后一个epoch保存模型
        if (epoch + 1) == NUM_EPOCHS:

            state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch + 1),
            }

            # 检查目录是否存在，如果不存在则创建
            if not os.path.exists(SAVE_DIR):
                os.makedirs(SAVE_DIR)

            # 保存模型状态
            torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch + 1))))

Epoch: 1/10 Train Loss: 0.0108 Acc: 0.4970
Epoch: 2/10 Train Loss: 0.0069 Acc: 0.6848
Begin test......
Test Loss: 0.0073 Acc: 0.6839
Epoch: 3/10 Train Loss: 0.0051 Acc: 0.7682
Epoch: 4/10 Train Loss: 0.0040 Acc: 0.8205
Begin test......
Test Loss: 0.0055 Acc: 0.7637
Epoch: 5/10 Train Loss: 0.0031 Acc: 0.8608
Epoch: 6/10 Train Loss: 0.0023 Acc: 0.8965
Begin test......
Test Loss: 0.0053 Acc: 0.7905
Epoch: 7/10 Train Loss: 0.0017 Acc: 0.9227
Epoch: 8/10 Train Loss: 0.0012 Acc: 0.9475
Begin test......
Test Loss: 0.0064 Acc: 0.7819
Epoch: 9/10 Train Loss: 0.0008 Acc: 0.9635
Epoch: 10/10 Train Loss: 0.0005 Acc: 0.9788
Begin test......
Test Loss: 0.0073 Acc: 0.7873


数据集加载与处理-daily climate

In [300]:
# 定义数据集的URL和存储路径
url = 'https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data'
data_dir = './dataset-climate'

# 创建存储路径（如果不存在）
os.makedirs(data_dir, exist_ok=True)

# 下载并解压缩数据集
filename = url.split('/')[-1]
filepath = os.path.join(data_dir, filename)

if not os.path.exists(filepath):
    urllib.request.urlretrieve(url, filepath)


In [308]:
train_df = pd.read_csv('dataset-climate/DailyDelhiClimateTrain.csv')
test_df = pd.read_csv('dataset-climate/DailyDelhiClimateTest.csv')
train_df.head()


,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [309]:
def humidity_pressure_ratio(df):
    df['humidity_pressure_ratio'] = df['humidity'] / df['meanpressure']
    return df

def get_date_columns(date):
    year, month, day = date.split('-')
    return (year, month, day)

train_df = humidity_pressure_ratio(train_df)
test_df = humidity_pressure_ratio(test_df)

tr_date_cols = train_df['date'].apply(get_date_columns)
te_date_cols = test_df['date'].apply(get_date_columns)

train_df[['year', 'month', 'day']] = pd.DataFrame(tr_date_cols.tolist(), index=train_df.index)
test_df[['year', 'month', 'day']] = pd.DataFrame(te_date_cols.tolist(), index=test_df.index)

train_df = train_df.drop('date',axis=1)
train_df = train_df.drop('year',axis=1)
print('处理后的训练集数据')
train_df.head()

处理后的训练集数据


,meantemp,humidity,wind_speed,meanpressure,humidity_pressure_ratio,month,day
0,10.000000,84.500000,0.000000,1015.666667,0.083197,01,01
1,7.400000,92.000000,2.980000,1017.800000,0.090391,01,02
2,7.166667,87.000000,4.633333,1018.666667,0.085406,01,03
3,8.666667,71.333333,1.233333,1017.166667,0.070129,01,04
4,6.000000,86.833333,3.700000,1016.500000,0.085424,01,05


In [343]:
tr_timeseries = train_df[['month', 'day', 'humidity', 'wind_speed', 'meanpressure', 'humidity_pressure_ratio', 'meantemp']].values.astype('float32')
te_timeseries = test_df[['month', 'day',  'humidity', 'wind_speed', 'meanpressure', 'humidity_pressure_ratio', 'meantemp']].values.astype('float32')

new = pd.concat([train_df, test_df], axis=0).reset_index().drop('index', axis=1)
new_timeseries = new[['month', 'day',  'humidity', 'wind_speed', 'meanpressure',  'humidity_pressure_ratio', 'meantemp']].values.astype('float32')

scaler = MinMaxScaler()
tr_timeseries = scaler.fit_transform(tr_timeseries)
te_timeseries = scaler.transform(te_timeseries)

def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[:,:6][i:i+lookback]
        target = dataset[:, 6][i:i+lookback]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 7

train, test = tr_timeseries, te_timeseries
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

X_train, X_test = X_train, X_test
y_train, y_test = y_train, y_test

loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train),
                         batch_size = 8, shuffle = True)

In [344]:
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = 6, 
                            num_layers = 2,
                            hidden_size = 128,  
                            batch_first = True, 
                            bidirectional= True)
        
        self.dropout = nn.Dropout(0.2)
        self.linear1 = nn.Linear(128*2, 64) 
        self.linear2 = nn.Linear(64, 8) 
        self.output_linear = nn.Linear(8, 1)
        
    def forward(self, x):  
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.output_linear(x)
        return x

In [348]:
model = LSTMModel()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)
loss_fn = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
Epoch = 200

In [349]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [350]:
for epoch in range(Epoch+1):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred.squeeze(), y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    
    with torch.no_grad():
        y_pred = model(X_train) 
        train_rmse = np.sqrt(loss_fn(y_pred, y_train.unsqueeze(2)))
        train_preds = y_pred.clone().detach().cpu().numpy()
        
        y_pred = model(X_test) 
        test_rmse = np.sqrt(loss_fn(y_pred, y_test.unsqueeze(2)))
        test_preds = y_pred.clone().detach().cpu().numpy()
        
        scheduler.step(test_rmse)
    if epoch % 20 == 0:
        print('Epoch', epoch,'/200', ': train RMSE: ', train_rmse.numpy(), ': test RMSE', test_rmse.numpy())

Epoch 0 /200 : train RMSE:  0.16631314 : test RMSE 0.13590318
Epoch 20 /200 : train RMSE:  0.060520027 : test RMSE 0.09534419
Epoch 00023: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00034: reducing learning rate of group 0 to 2.5000e-04.
Epoch 40 /200 : train RMSE:  0.0616768 : test RMSE 0.08492934
Epoch 00045: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00056: reducing learning rate of group 0 to 6.2500e-05.
Epoch 60 /200 : train RMSE:  0.055089902 : test RMSE 0.08253997
Epoch 00067: reducing learning rate of group 0 to 3.1250e-05.
Epoch 00078: reducing learning rate of group 0 to 1.5625e-05.
Epoch 80 /200 : train RMSE:  0.054498214 : test RMSE 0.08075174
Epoch 00089: reducing learning rate of group 0 to 7.8125e-06.
Epoch 00100: reducing learning rate of group 0 to 3.9063e-06.
Epoch 100 /200 : train RMSE:  0.054426435 : test RMSE 0.080931954
Epoch 00111: reducing learning rate of group 0 to 1.9531e-06.
Epoch 120 /200 : train RMSE:  0.05438379 : test RMSE 0.08070